title: Overdraft Decision Investigation
author: Helder Silva 
date: 2020-11-06
region: EU  
tags: overdraft, bank products, platform, einsteinium, plutonium, credit scores, schufa, write-offs
summary: - 65% of the Einsteinium checks don't lead to an overdraft status change in the same day (out of these, 31% of the status changes happened up to 5 days after the Einsteinium check day). Our main assumption here is that, after the Einsteinium check, users may be asked to complete a step in the app to get their overdraft status change, which would explain some of this delay. <br>
- 1% of the overdrafts generated before Einsteinium were written-off. This ratio decreases to 0.6% after the implementation of Einsteinium - this decrease is mainly driven by our internal scores write-off ratio of 0.16%, the lowest ratio of all our providers so far.<br>
- Decisions made with Schufa scores have been decreasing since June 2020, taking 38% of the generated overdraft decisions in October 2020, whereas the use of our internal scores has been increasing since May 2020, leading to 56% of the generated overdraft decisions.<br>
- Moving forward we will need an ID to match the Einsteinium scores with the Plutonium decision to determine which exact Einsteinium result led to which Plutonium decision.<br>


<div class="alert alert-block alert-success">
    <H1>Overdraft Decision Investigation</H1>

</div>

In this investigation we aimed to replicate the communication between Einsteinium and Plutonium in the Overdraft decision making - you can find more information on how these services communicate [here](https://number26-jira.atlassian.net/wiki/spaces/NTD/pages/1584366242/How+does+Einsteinium+work+-+New+credit+risk+parameters+service). For this we looked into all available data in these services up until October 31st 2020. 

Here are our main findings:
- 65% of the Einsteinium checks don't lead to an overdraft status change in the same day (out of these, 31% of the status changes happened up to 5 days after the Einsteinium check day). Our main assumption here is that, after the Einsteinium check, users may be asked to complete a step in the app to get their overdraft status change, which would explain some of this delay. 
- 1% of the overdrafts generated before Einsteinium were written-off. This ratio decreases to 0.6% after the implementation of Einsteinium - this decrease is mainly driven by our internal scores write-off ratio of 0.16%, the lowest ratio of all our providers so far.
- Decisions made with Schufa scores have been decreasing since June 2020, taking 38% of the generated overdraft decisions in October 2020, whereas the use of our internal scores has been increasing since May 2020, leading to 56% of the generated overdraft decisions.
- Moving forward we will need an ID to match the Einsteinium scores with the Plutonium decision to determine which exact Einsteinium result led to which Plutonium decision.


In [1]:
import pandas as pd
import numpy as np
import altair as alt
from utils.datalib_database import df_from_sql

import utils.altair_functions as af

In [2]:
query_eligibility = """
--credit scores per user per requested date
with scores_per_day as (
select 
user_created,
created::date as created_date,
max(credit_score_credit_bureau_score_date) as max_bureau_score_date, -- there are multiple ones here
min(credit_score_score::int) as min_credit_score,
max(credit_score_score::int) as max_credit_score,
listagg(distinct credit_score_provider, ', ') within group (order by credit_score_provider)  as provider_list,
count(case when credit_score_provider = 'N26' and credit_score_credit_bureau_score_date::date >= created::date - interval '30 day' then 1 end) as valid_n26_score_count,
count(case when credit_score_provider = 'N26' then 1 end) as all_n26_score_count,
count(case when credit_score_provider = 'SCHUFA' then 1 end) as schufa_score_count,
count(case when credit_score_provider = 'CRIF' then 1 end) as crif_score_count,
count(case when credit_score_provider = 'EXPERIAN' then 1 end) as experian_score_count,
count(case when credit_score_provider is null then 1 end) as null_score_count
from private.es_requested_credit_score ercs 
inner join dbt.zrh_users on credit_score_user_id = user_id
group by 1, 2
),
eligibility_per_day as (
select 
user_created,
created::date as created_date,
count(case when eligibility_result = true then 1 end) as positive_results,
count(case when eligibility_result = false then 1 end) as negative_results
from private.es_requested_eligibility ere
inner join dbt.zrh_users on eligibility_user_id = user_id
group by 1, 2
),
pu_generated as (
select 
user_created,
created::date as created_date,
status
from pu_overdraft_history poh 
where status in ('GENERATED', 'ENABLED', 'DISABLED')
group by 1, 2, 3
)
select *,
case when positive_results =0 and negative_results >0 then 'has only negative es results'
when positive_results >0 and negative_results =0 then 'has only positive es results'
when positive_results >0 and negative_results >0 then 'has both positive and negative es results'
else 'other' end as es_result_type,
coalesce(status, 'NO OD FOUND') as od_status,
case when spd.min_credit_score is not null then true else false end as has_credit_score,
case when pg.user_created is not null then true else false end as has_generated_od,
case when min_credit_score != max_credit_score then true else false end as had_different_scores,
case when min_credit_score is null then 'doesnt have a credit score'
when min_credit_score::int <= 12 and max_credit_score::int <= 12 then 'has positive credit scores only'
when min_credit_score::int >= 13 and max_credit_score::int >= 13 then 'has negative credit scores only'
when min_credit_score::int <= 12 and max_credit_score::int >= 13 then 'has both positive and negative credit scores'
else 'other' end as credit_score_type
from eligibility_per_day epd
left join scores_per_day spd using (user_created, created_date) 
left join pu_generated pg using (user_created, created_date) 
where created_date <= '2020-10-31'
order by 5 desc
"""

In [3]:
query_od_status = """
with scores_per_day as (
select 
user_created,
created::date as created_date,
max(credit_score_credit_bureau_score_date) as max_bureau_score_date, -- there are multiple ones here
min(credit_score_score::int) as min_credit_score,
max(credit_score_score::int) as max_credit_score,
listagg(distinct credit_score_provider, ', ') within group (order by credit_score_provider)  as provider_list,
count(case when credit_score_provider = 'N26' and credit_score_credit_bureau_score_date::date >= created::date - interval '30 day' then 1 end) as valid_n26_score_count,
count(case when credit_score_provider = 'N26' then 1 end) as all_n26_score_count,
count(case when credit_score_provider = 'SCHUFA' then 1 end) as schufa_score_count,
count(case when credit_score_provider = 'CRIF' then 1 end) as crif_score_count,
count(case when credit_score_provider = 'EXPERIAN' then 1 end) as experian_score_count,
count(case when credit_score_provider is null then 1 end) as null_score_count
from private.es_requested_credit_score ercs 
inner join dbt.zrh_users on credit_score_user_id = user_id
group by 1, 2
),
eligibility_per_day as (
select 
user_created,
created::date as created_date,
count(case when eligibility_result = true then 1 end) as positive_results,
count(case when eligibility_result = false then 1 end) as negative_results
from private.es_requested_eligibility ere
inner join dbt.zrh_users on eligibility_user_id = user_id
group by 1, 2
),
pu_generated as (
select 
user_created,
created::date as created_date,
status
from pu_overdraft_history poh 
where status in ('GENERATED', 'ENABLED', 'DISABLED')
group by 1, 2, 3
), scores_overview  as (
select *,
case when positive_results =0 and negative_results >0 then 'has only negative es results'
when positive_results >0 and negative_results =0 then 'has only positive es results'
when positive_results >0 and negative_results >0 then 'has both positive and negative es results'
else 'other' end as es_result_type,
coalesce(status, 'NO OD FOUND') as od_status,
case when spd.min_credit_score is not null then true else false end as has_credit_score,
case when pg.user_created is not null then true else false end as has_generated_od,
case when min_credit_score != max_credit_score then true else false end as had_different_scores,
case when min_credit_score is null then 'doesnt have a credit score'
when min_credit_score::int <= 12 and max_credit_score::int <= 12 then 'has positive credit scores only'
when min_credit_score::int >= 13 and max_credit_score::int >= 13 then 'has negative credit scores only'
when min_credit_score::int <= 12 and max_credit_score::int >= 13 then 'has both positive and negative credit scores'
else 'other' end as credit_score_type
from eligibility_per_day epd
left join scores_per_day spd using (user_created, created_date) 
left join pu_generated pg using (user_created, created_date) 
order by 5 desc
),
totals as  (
select
user_created,
max(created_date) as max_created_date
from scores_overview 
where credit_score_type = 'has positive credit scores only' 
and es_result_type = 'has only positive es results'
and od_status = 'NO OD FOUND'
group by 1
), joins as (
select 
'previous status' as label,
t.user_created,
max_created_date,
etl_updated,
status,
row_number() over (partition by t.user_created order by etl_updated desc) as rn
from totals t
inner join pu_overdraft_history poh 
on poh.user_created = t.user_created 
and poh.etl_updated::date < max_created_date
union all 
select 
'next status' as label,
t.user_created,
max_created_date,
etl_updated,
status,
row_number() over (partition by t.user_created order by etl_updated) as rn
from totals t
inner join pu_overdraft_history poh 
on poh.user_created = t.user_created 
and poh.etl_updated::date > max_created_date
)
select 
label,
user_created, 
to_char(etl_updated, 'YYYY-MM-DD') as status_date, 
to_char(max_created_date, 'YYYY-MM-DD') as decision_date,
date_diff('day', max_created_date::date, etl_updated::date) as diff,
status
from joins 
inner join dbt.zrh_users using (user_created)
where rn =1
and max_created_date <= '2020-10-31'
"""

In [35]:
df = df_from_sql("redshiftreader", query_eligibility)

In [5]:
df_eligibility = df

# Overdraft Decision Overview

In order to replicate the overdraft decision, we will be looking into: 
- the Einsteinuim eligibility results per day per users, 
- their credit scores in that day 
- and whether they had an overdraft status change (GENERATED, ENABLED or DISABLED).

It seems that the majority (64.8%) of the Einsteinium checks don't lead to an overdraft status change in the same day. We also found 278 cases with both positive and negative Einsteinium results and 547 cases with both positive and negative credit scores in the same day. This means that moving forward we will need an ID to make the match between Einsteinium and Plutonium to determine which exact Einsteinium result led to which Plutonium decision. 

In [6]:
df_result_type = (
    df_eligibility.groupby(["es_result_type"])
    .count()
    .reset_index()
    .sort_values(by=["user_created"], ascending=False)
)
df_score_type = (
    df_eligibility.groupby(["credit_score_type"])
    .count()
    .reset_index()
    .sort_values(by=["user_created"], ascending=False)
)
df_od_status = (
    df_eligibility.groupby(["od_status"])
    .count()
    .reset_index()
    .sort_values(by=["user_created"], ascending=False)
)

df_result_type[["percentage_es_results"]] = df_result_type[["user_created"]].apply(
    lambda x: round((x / x.sum()) * 100, 1), axis=0
)
df_score_type[["percentage_es_scores"]] = df_score_type[["user_created"]].apply(
    lambda x: round((x / x.sum()) * 100, 1), axis=0
)
df_od_status[["percentage_od_changes"]] = df_od_status[["user_created"]].apply(
    lambda x: round((x / x.sum()) * 100, 1), axis=0
)

result_type = af.column_single_label(
    df_result_type,
    af.petrol,
    "es_result_type:N",
    "percentage_es_results:Q",
    250,
    400,
    "-y",
)
score_type = af.column_single_label(
    df_score_type,
    af.wheat,
    "credit_score_type:N",
    "percentage_es_scores:Q",
    250,
    400,
    "-y",
)
od_status = af.column_single_label(
    df_od_status, af.teal, "od_status:N", "percentage_od_changes:Q", 250, 400, "-y"
)
result_type | score_type | od_status

alt.HConcatChart(...)

## Breakdown per Einsteinium result type, credit score and overdraft status

In [7]:
df_overview = (
    df_eligibility.groupby(["es_result_type", "credit_score_type", "od_status"])
    .count()
    .reset_index()
    .sort_values(by=["user_created"], ascending=False)
)
df_overview = df_overview[
    ["es_result_type", "credit_score_type", "od_status", "user_created"]
]
df_overview[["percentage"]] = df_overview[["user_created"]].apply(
    lambda x: round((x / x.sum()) * 100, 1), axis=0
)
df_overview

,es_result_type,credit_score_type,od_status,user_created,percentage
23,has only positive es results,has positive credit scores only,NO OD FOUND,140668,25.8
19,has only positive es results,has negative credit scores only,NO OD FOUND,118879,21.8
9,has only negative es results,doesnt have a credit score,NO OD FOUND,91344,16.7
20,has only positive es results,has positive credit scores only,DISABLED,76732,14.1
21,has only positive es results,has positive credit scores only,ENABLED,57624,10.6
22,has only positive es results,has positive credit scores only,GENERATED,40330,7.4
6,has only negative es results,doesnt have a credit score,DISABLED,10256,1.9
16,has only positive es results,has negative credit scores only,DISABLED,6169,1.1
11,has only positive es results,doesnt have a credit score,NO OD FOUND,2273,0.4
12,has only positive es results,has both positive and negative credit scores,DISABLED,293,0.1


For the cases where we found a Plutonium decision on the same day of the Einsteinium (es) checks, we found some interesting cases that we should look into and decide whether these are expected behaviour or not.
- has only positive es results, has positive credit scores only, NO OD FOUND IN DAY
- has only positive es results, has positive credit scores only, DISABLED
- has only negative es results, doesn't have a credit score, ENABLED
- has only positive es results, has negative credit scores only, ENABLED
- has only positive es results, has both positive and negative credit scores, ENABLED
- has both positive and negative es results, has positive credit scores only, ENABLED
- has only positive es results, has negative credit scores only, GENERATED	

In [36]:
# users with different scores on the same day
df_scores_overview = df_eligibility[
    (df_eligibility.had_different_scores == True)
    & (df_eligibility.has_generated_od == True)
    & (
        (df_eligibility.od_status == "ENABLED")
        | (df_eligibility.od_status == "GENERATED")
    )
]
df_scores_overview["min_credit_score"] = df_scores_overview["min_credit_score"].astype(
    int
)
df_scores_overview["max_credit_score"] = df_scores_overview["max_credit_score"].astype(
    int
)
df_scores_overview["score_diff"] = (
    df_scores_overview["min_credit_score"].astype(str)
    + " , "
    + df_scores_overview["max_credit_score"].astype(str)
)

# Credit Scores Divergence

Here is an overview of users that have both positive and negative credit scores on the day of a positive Plutonium decision (ENABLED or GENERATED). Once we have an ID to match Einsteinium and Plutonium, we should check which of these scores led to the Overdraft decision (even though we have a low count of users here, these should be looked into to make sure we are not enabling/ generating overdrafts for users with a negative credit score).

In [9]:
df_scores_overview = (
    df_scores_overview.groupby(["score_diff", "credit_score_type"])
    .count()
    .reset_index()
    .sort_values(by=["user_created"], ascending=False)
)
df_scores_overview["count_od_generated_or_enabled"] = df_scores_overview["user_created"]
df_scores_overview = df_scores_overview[
    ["score_diff", "credit_score_type", "count_od_generated_or_enabled"]
]
df_scores_overview = df_scores_overview[
    (
        df_scores_overview.credit_score_type
        == "has both positive and negative credit scores"
    )
]
df_scores_overview

,score_diff,credit_score_type,count_od_generated_or_enabled
9,"12 , 13",has both positive and negative credit scores,40
6,"11 , 14",has both positive and negative credit scores,25
10,"12 , 14",has both positive and negative credit scores,20
2,"10 , 13",has both positive and negative credit scores,13
30,"9 , 14",has both positive and negative credit scores,11
3,"10 , 14",has both positive and negative credit scores,11
23,"8 , 14",has both positive and negative credit scores,8
11,"12 , 15",has both positive and negative credit scores,5
29,"9 , 13",has both positive and negative credit scores,4
5,"11 , 13",has both positive and negative credit scores,3


# Why are there so many cases with Einsteinium checks but no overdraft decision on the same day?

Since we found so many situations of Einsteinium checks without an overdraft decision in the same day, we decided to have a deeper look into this one. We started by comparing which overdraft status are more common before and after the Einsteinium checks day. The main pattern we found here is that the status Blocked, Blocked Requested, Expired, not Granted and Override Request only show after the day of the Einsteinium checks, which could mean that users with this status are not being re-evaluated by Einsteinium.

In [37]:
df_od_status = df_from_sql("redshiftreader", query_od_status)

In [39]:
df_count_per_label = (
    df_od_status.groupby(["status", "label"])
    .count()
    .reset_index()
    .sort_values(by=["diff"], ascending=False)
)

df_count_per_prev_label = df_count_per_label[
    (df_count_per_label.label == "previous status")
]
df_count_per_prev_label[["percentage"]] = df_count_per_prev_label[
    ["user_created"]
].apply(lambda x: round((x / x.sum()) * 100, 1), axis=0)

df_count_per_next_label = df_count_per_label[
    (df_count_per_label.label == "next status")
]
df_count_per_next_label[["percentage"]] = df_count_per_next_label[
    ["user_created"]
].apply(lambda x: round((x / x.sum()) * 100, 1), axis=0)

df_count_per_label = pd.concat([df_count_per_prev_label, df_count_per_next_label])
df_count_per_label["status_timing"] = df_count_per_label["label"]
df_count_per_label = df_count_per_label[
    ["status", "status_timing", "user_created", "percentage"]
]

In [12]:
af.square_multi(
    df_count_per_label, "percentage:O", "status:N", "status_timing:N", 800, 200, 2000
).properties(title="Percentage of Overdraft Status per Status Timing")

alt.VConcatChart(...)

In [40]:
df_od_status_grouped = (
    df_od_status.groupby(["label", "diff"])
    .count()
    .reset_index()
    .sort_values(by=["diff"])
)
df_prev_status = df_od_status_grouped[(df_od_status_grouped.label == "previous status")]
df_next_status = df_od_status_grouped[(df_od_status_grouped.label == "next status")]
df_prev_status["percent_status_changes"] = (
    df_prev_status["status"].cumsum() / df_prev_status["status"].sum()
) * 100
df_next_status["percent_status_changes"] = (
    df_next_status["status"].cumsum() / df_next_status["status"].sum()
) * 100

We checked when the users had an overdraft status change before the Einsteinium check day, and the main pattern we found here is that about 25% of the status changes happened up to 20 days before the Einsteinium check day.

In [33]:
af.line_single(
    df_prev_status, af.petrol, "diff:O", "percent_status_changes:Q", 800, 400, "x"
).properties(title="Last Overdraft History Change before Einsteinium Decision")

alt.Chart(...)

We also looked into when the users had an overdraft status change after the Einsteinium check day, and about 31% of the status changes happened up to 5 days after the Einsteinium check day. Our main assumption here is that, after the Einsteinium check, users may be asked to complete a step in the app to get their overdraft status change, which would explain some of this delay. In order to explore this topic further, we will need the ID match between Einsteinium and Plutonium to see the full timeline for each decision.

In [34]:
af.line_single(
    df_next_status, af.petrol, "diff:O", "percent_status_changes:Q", 800, 400, "x"
).properties(title="Next Overdraft History Change after Einsteinium Decision")

alt.Chart(...)

# Generated Overdrafts by Einsteinium Provider
Here we focus on understanding: 
- the dates of generated overdraft for each user, 
- if these users went through the einsteinium eligibility on that day, 
- if these users have a write-off

In some cases, we can find more than one score provider in a day, we will be able to determine which provider led to which specific user once we have the ID match between Einsteinium and Plutonium.

In [16]:
query_od_output = """
with es_eligibilty as (
select 
user_created, 
created::date as created_date
from private.es_requested_eligibility
inner join dbt.zrh_users on eligibility_user_id = user_id
group by 1, 2
),
es_scores as (
select 
user_created,
created::date as created_date,
listagg(distinct credit_score_provider, ', ') within group (order by credit_score_provider)  as provider_list
from private.es_requested_credit_score ercs 
inner join dbt.zrh_users on credit_score_user_id = user_id
group by 1, 2
),
generated_od as (
select 
user_created,
created::date as created_date
from pu_overdraft_history po 
where status = 'GENERATED'
group by 1, 2
),
arrears_txn as (
select user_created,
max(arrears_update_tstmp) as max_arrears_update_tstamp
from dbt.arrears_transactions t 
where reason = 'Arranged Overdraft'
group by 1
),
write_offs as (
select user_created,
max(write_off_dt) as write_off_dt
from dbt.write_off wo  
where reason = 'Arranged Overdraft'
group by 1
)
select 
date_trunc('month', o.created_date) as month,
coalesce(provider_list, 'NOT IN EINSTEINIUM') as provider_list,
count(distinct o.user_created) as generated_od_users,
count(distinct e.user_created) as eligib_users,
count(distinct e.user_created) as scores_users,
generated_od_users - eligib_users as diff_od_eligib,
count(case when t.max_arrears_update_tstamp <= created_date then 1 end) as arrears_before_od_gen,
count(case when t.max_arrears_update_tstamp > created_date then 1 end) as arrears_after_od_gen,
count(case when wo.user_created is not null then 1 end) as has_write_off
from generated_od o
left join es_eligibilty e
using (user_created, created_date)
left join es_scores s
using (user_created, created_date)
left join arrears_txn t 
using(user_created)
left join write_offs wo 
using(user_created)
where created_date <= '2020-10-31'
group by 1, 2
"""

In [41]:
df_od_output = df_from_sql("redshiftreader", query_od_output)

In order to define a write-off ratio per provider, we divided the number of generated overdrafts per provider by the number of write-offs for these overdrafts. Through this metric we can find that 1% of the overdrafts generated before Einsteinium are written-off, and that this ratio decreases to 0.6% after the implementation of Einsteinium. This decrease is mainly driven by our internal scores write-off ratio of 0.16%, the lowest ratio of all our providers so far.

In [20]:
df_od_output_grouped = (
    df_od_output.groupby(["provider_list"])
    .sum()
    .reset_index()
    .sort_values(by=["generated_od_users"], ascending=False)
)
df_od_output_grouped["write_off_ratio(%)"] = (
    round(
        df_od_output_grouped["has_write_off"]
        / df_od_output_grouped["generated_od_users"],
        4,
    )
    * 100
)
df_od_output_grouped[
    ["provider_list", "generated_od_users", "has_write_off", "write_off_ratio(%)"]
]

,provider_list,generated_od_users,has_write_off,write_off_ratio(%)
4,NOT IN EINSTEINIUM,170961,1706,1.00
5,SCHUFA,22059,198,0.90
2,N26,14033,24,0.17
0,CRIF,2497,15,0.60
3,"N26, SCHUFA",93,0,0.00
1,"CRIF, N26",4,0,0.00


In [42]:
df_od_output_chart = df_od_output[(df_od_output.month >= "2019-10-01")]
df_od_output_chart["month"] = df_od_output["month"].astype(str)

Finally, we can see that since the implementation of Einsteinium, decisions made with Schufa scores have been decreasing since June 2020, taking 38% of the generated overdraft decisions in October 2020, whereas the use of our internal scores has been increasing since May 2020, leading to 56% of the generated overdraft decisions. 

In [22]:
af.line_multi(
    df_od_output_chart,
    "provider_list:N",
    "month:O",
    "generated_od_users:Q",
    800,
    400,
    "x",
).properties(title="Einsteinium Score providers per month")

alt.Chart(...)